# Send data (easy)
This task will set up a simple network for sending data into a neural network simulation during execution.

Setup a function to be executed at start up, accepting a label and a connection.  Using sleep, this should send a spike into the network approximately every 100ms for a total of 1000ms. 

In [ ]:
from time import sleep

def send_spikes(label, connection):
    for i in range(10):
        sleep(0.1)
        connection.send_spike(label, 0)

Set up a ```SpynnakerLiveSpikesConnection``` with a send label of ```sender``` and local port of None.

In [ ]:
import spynnaker.pyNN as sim

connection = sim.external_devices.SpynnakerLiveSpikesConnection(local_port=None, send_labels=["sender"])

Register the function created previously to be called for the label ```sender``` when the simulation starts.

In [ ]:
connection.add_start_resume_callback("sender", send_spikes)

Setup the simulation with a timestep of 1.0ms.

In [ ]:
sim.setup(1.0)

Create a Population of 100 LIF neurons and record the spikes of the Population.

In [ ]:
pop = sim.Population(100, sim.IF_curr_exp(), label="pop")
pop.record("spikes")

Create a Population of 1 ```SpikeInjector``` with a label of ```sender```.  Remember to register the connection.

In [ ]:
sender = sim.Population(1, sim.external_devices.SpikeInjector(database_notify_port_num=connection.local_port), label="sender")

Connect the population to itself randomly with a fixed probability of 0.08, a weight of 0.5nA and a delay of 2ms.

In [ ]:
sim.Projection(pop, pop, sim.FixedProbabilityConnector(0.08), sim.StaticSynapse(weight=0.5, delay=2))

Connect the sender to all of the neurons in the Population with a weight of 5nA and a delay of 1ms.

In [ ]:
sim.Projection(sender, pop, sim.AllToAllConnector(), sim.StaticSynapse(weight=5, delay=1))

Run the simulation for 1000ms.

In [ ]:
sim.run(1000)

Extract the spikes and end the simulation.

In [ ]:
spikes = pop.get_data("spikes").segments[0].spiketrains
sim.end()

Plot the spikes.

In [ ]:
import pyNN.utility.plotting as plot

plot.Figure(
    plot.Panel(spikes, yticks=True, markersize=5, xlim=(0, 1000)),
    title="Spikes"
)

## Extension 1 (Moderate)
Try having one ```SpikeInjector``` neuron for each of the Population neurons, changing the connector to a OneToOneConnector, and then send random spikes from the function.

In [ ]:
from time import sleep
import random

def send_spikes(label, connection):
    for i in range(100):
        sleep(0.01)
        neuron_id = random.randint(0, 99)
        connection.send_spike(label, neuron_id)
        
import spynnaker.pyNN as sim

connection = sim.external_devices.SpynnakerLiveSpikesConnection(local_port=None, send_labels=["sender"])
connection.add_start_resume_callback("sender", send_spikes)
sim.setup(1.0)
pop = sim.Population(100, sim.IF_curr_exp(), label="pop")
pop.record("spikes")
sender = sim.Population(100, sim.external_devices.SpikeInjector(database_notify_port_num=connection.local_port), label="sender")
sim.Projection(pop, pop, sim.FixedProbabilityConnector(0.08), sim.StaticSynapse(weight=0.5, delay=2))
sim.Projection(sender, pop, sim.OneToOneConnector(), sim.StaticSynapse(weight=5, delay=1))
sim.run(1000)
spikes = pop.get_data("spikes").segments[0].spiketrains
sim.end()
import pyNN.utility.plotting as plot

plot.Figure(
    plot.Panel(spikes, yticks=True, markersize=5, xlim=(0, 1000)),
    title="Spikes"
)